# Access to ECMWF data

Access ECMWF Open Data: https://www.ecmwf.int/en/forecasts/datasets/open-data \
Forecasted variables can be accessed through `ecwmf.opendata`

## Setup

In [9]:
import datetime
import eccodes
from ecmwf.opendata import Client
import os
import numpy as np
import pdbufr
import pygrib

The user will select the timestep from the widget, so date/time and step needed to retrieve ecmwf data will have to be provided as variables. Below I'm setting them as tool variables. Date and time can be provided in different styles: https://github.com/ecmwf/ecmwf-opendata/tree/main#date-and-time

In [3]:
# Tool variables
date = '2023-07-30'
time = 0 #available times: 0, 6, 12, 18
step = 48 #hours

# Forecast

## 0. General retrieval process using ecmwf.opendata

```python
client = Client(source="ecmwf")

request = {
    "time": ,
    "stream": ,
    "type": , #type of data, default is fc
    "step": ,
    "date": ,
    "param": , #meteorological parameters. multiple parameters can be specified in a list.
}

client.retrieve(request = request, target = "") #takes request as input and will retrieve the corresponding data from the server and write them in the user's target file
client.download(request = request, target = "") #will download the whole data files from the server, ignoring keywords like param, levelist or number
client.latest(request = request, target = "")   #returns the date of the most recent matching forecast without downloading the data
```

To obtain data to be used in the dashboard, it will have to be retrieved through `Client.retrieve`, then uploaded in the dashboard.

More info on request parameters: https://github.com/ecmwf/ecmwf-opendata/tree/main#request-keywords \
List of available parameters: https://github.com/ecmwf/ecmwf-opendata/tree/main#parameters-and-levels

Tutoral on GRIB files: https://confluence.ecmwf.int/display/CKB/How+to+plot+GRIB+files+with+Python+and+matplotlib \

## 1. Cyclone characteristics
Tutorial on the access to cyclone characteristics data: TropiDash/notebooks-examples-trials/Inspect_tc_bufr_data.ipynb

In [25]:
# Download the Tropical Cyclone tracks from ECMWF's 00UTC ENS forecast
client.retrieve(
    time=0,
    stream="enfo",
    type="tf",
    step=240,
    target="track_data/ens_tracks.bufr",
)

20230727000000-240h-enfo-tf.bufr:   0%|          | 0.00/535k [00:00<?, ?B/s]

In [ ]:
# Import bufr file of the TC tracks
df_tracks = pdbufr.read_bufr("track_data/ens_tracks.bufr",
    columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
             "pressureReducedToMeanSeaLevel",))
storms = df_tracks.stormIdentifier.unique()
df.head()

In [3]:
# Cyclone temporal horizon definition
# For DOKSURI forecast of the 25th July we have 41 snapshot with 6h step
start_dte = datetime(2023, 7, 25)
end_dte = datetime(2023, 8, 4)
step = timedelta(hours=6)
time_steps = pd.date_range(start_dte, end_dte, freq=step)

In [4]:
# Save all ensemble members track in a list called locations
locations = []
tracks_len = []
tracks_date = []
for m in members:
    dft = df[df.ensembleMemberNumber == m]
    dft_noNaN = dft.dropna()
    lat = dft_noNaN.latitude.values
    lon = dft_noNaN.longitude.values
    locs = [[lat[i], lon[i]] for i in range(len(lat))]
    track_len = len(locs)
    date_range = time_steps[:track_len]
    locations.append(locs)
    tracks_len.append(track_len)
    tracks_date.append(date_range)

## 2. Atmosphere and ocean characteristics
From ECMWF opendata, only forecasted data can be retrieved, not observed data

In [27]:
# Variable: Wind speed
# Download from ECMWF open data
request = {
    "time": time,
    # "stream": "fc", #HRES
    "type": "fc",
    "step": step,
    # "date": date,
    "param": ["u", "v"],
}

client = Client(source = "ecmwf")
client.retrieve(request = request, target = "test_data_access/wind.grib2")

20230730000000-48h-oper-fc.grib2:   0%|          | 0.00/5.11M [00:00<?, ?B/s]

### pygrib
Opening a GRIB with pygrib results in a list of "messages" accessible throught the .select module \
PyGRIB documentation: https://jswhit.github.io/pygrib/api.html#example-usage

In [8]:
grbs = pygrib.open("test_data_access/wind.grib2")
grbs.select()[0]

1:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 30000 Pa:fcst time 48 hrs:from 202307300000

In [6]:
for grb in grbs:
    print(grb)

1:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 30000 Pa:fcst time 48 hrs:from 202307300000
2:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 30000 Pa:fcst time 48 hrs:from 202307300000
3:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 5000 Pa:fcst time 48 hrs:from 202307300000
4:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 5000 Pa:fcst time 48 hrs:from 202307300000
5:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 25000 Pa:fcst time 48 hrs:from 202307300000
6:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 25000 Pa:fcst time 48 hrs:from 202307300000
7:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 20000 Pa:fcst time 48 hrs:from 202307300000
8:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 20000 Pa:fcst time 48 hrs:from 202307300000
9:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 92500 Pa:fc

### xarray
Tutorial on opening GRIB files with xarray: https://docs.xarray.dev/en/stable/examples/ERA5-GRIB-example.html

### Ocean variables

In [ ]:
stream = "wave" #ocean characteristics

## 3. Impact variables